# Interactions - Lab

## Introduction

In this lab, you'll explore interactions in the Boston Housing data set.

## Objectives

You will be able to:
- Understand what interactions are
- Understand how to accommodate for interactions in regression

## Build a baseline model 

You'll use a couple of built-in functions, which we imported for you below.

In [1]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.datasets import load_boston
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json

Import the Boston data set using `load_boston()`. We won't bother to preprocess the data in this lab. If you still want to build a model in the end, you can do that, but this lab will just focus on finding meaningful insights in interactions and how they can improve $R^2$ values.

In [2]:
regression = LinearRegression()
boston = load_boston()

Create a baseline model which includes all the variables in the Boston housing data set to predict the house prices. The use 10-fold cross-validation and report the mean $R^2$ value as the baseline $R^2$.

In [3]:
y = pd.DataFrame(boston.target, columns=['target'])
X = pd.DataFrame(boston.data, columns=boston.feature_names)
data = pd.concat([y, X], axis=1)

In [7]:
cv = KFold(n_splits=10, shuffle=True, random_state=1)
baseline = np.mean(cross_val_score(regression, X, y, scoring="r2", cv=cv))
baseline

0.7190106820189475

## See how interactions improve your baseline

Next, create all possible combinations of interactions, loop over them and add them to the baseline model one by one to see how they affect the R^2. We'll look at the 3 interactions which have the biggest effect on our R^2, so print out the top 3 combinations.

You will create a for loop to loop through all the combinations of 2 predictors. You can use `combinations` from itertools to create a list of all the pairwise combinations. To find more info on how this is done, have a look [here](https://docs.python.org/2/library/itertools.html).

In [5]:
from itertools import combinations
combinations = list(combinations(boston.feature_names, 2))

In [8]:
## code to find top 3 interactions by R^2 value here
comb_scores = []
for combination in combinations:
    # comb_score = mean R^2 value from cross validation test runs
    comb_score = np.mean(cross_val_score(regression, X[list(combination)], y, scoring="r2", cv=cv))
    comb_scores.append({'score': comb_score, 'combination': combination})
    
comb_scores[0]

{'score': 0.22109803800764474, 'combination': ('CRIM', 'ZN')}

In [12]:
sorted(comb_scores, key=lambda x: x['score'], o)

[{'score': 0.07315287063031399, 'combination': ('CHAS', 'DIS')},
 {'score': 0.10372229759051114, 'combination': ('CHAS', 'B')},
 {'score': 0.108650070259969, 'combination': ('ZN', 'DIS')},
 {'score': 0.12006325087303202, 'combination': ('DIS', 'B')},
 {'score': 0.1360080778439342, 'combination': ('AGE', 'DIS')},
 {'score': 0.13885784425014605, 'combination': ('ZN', 'CHAS')},
 {'score': 0.14204312222476245, 'combination': ('DIS', 'RAD')},
 {'score': 0.1452813960362018, 'combination': ('CRIM', 'CHAS')},
 {'score': 0.14847789627744845, 'combination': ('CRIM', 'DIS')},
 {'score': 0.15900077492464254, 'combination': ('CHAS', 'RAD')},
 {'score': 0.16062057713937467, 'combination': ('ZN', 'AGE')},
 {'score': 0.17069091592865057, 'combination': ('CHAS', 'AGE')},
 {'score': 0.1707770423922817, 'combination': ('CRIM', 'B')},
 {'score': 0.17214482625402336, 'combination': ('RAD', 'B')},
 {'score': 0.17394606907594112, 'combination': ('CRIM', 'RAD')},
 {'score': 0.18545603288526738, 'combination':

## Look at the top 3 interactions: "RM" as a confounding factor

The top three interactions seem to involve "RM", the number of rooms as a confounding variable for all of them. Let's have a look at interaction plots for all three of them. This exercise will involve:

- splitting our data up in 3 groups: one for houses with a few rooms, one for houses with a "medium" amount of rooms, one for a high amount of rooms.
- Create a function `build_interaction_rm`. This function takes an argument `varname` (which can be set equal to the column name as a string) and a column `description` (which describes the variable or varname, to be included on the x-axis of the plot). The function outputs a plot that uses "RM" as a confounding factor. 

We split the data set for high, medium and low amount of rooms for you.

In [ ]:
rm = np.asarray(df[["RM"]]).reshape(len(df[["RM"]]))

In [ ]:
high_rm = all_data[rm > np.percentile(rm, 67)]
med_rm = all_data[(rm > np.percentile(rm, 33)) & (rm <= np.percentile(rm, 67))]
low_rm = all_data[rm <= np.percentile(rm, 33)]

Create `build_interaction_rm`.

In [ ]:
def build_interaction_rm(varname, description):
    pass

Next, use build_interaction_rm with the three variables that came out with the highest effect on $R^2$

In [ ]:
# first plot

In [ ]:
# second plot

In [ ]:
# third plot

## Build a final model including all three interactions at once

Use 10-fold crossvalidation.

In [ ]:
# code here

In [ ]:
# code here

Our $R^2$ has increased considerably! Let's have a look in statsmodels to see if all these interactions are significant.

In [ ]:
# code here

What is your conclusion here?

In [ ]:
# formulate your conclusion

## Summary

You now understand how to include interaction effects in your model!